<a href="https://colab.research.google.com/github/lantrungseo/FUM-code/blob/main/Codes/FUM-train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
!ls '/content/gdrive/MyDrive/Dataset/MIND/_data_'
!ls '/content/gdrive/MyDrive/Dataset/MIND/_embeddings_/'
!ls '/content/gdrive/MyDrive/Projects/MIND-FUM'

!rsync -a '/content/gdrive/MyDrive/Projects/MIND-FUM/' .

MINDlarge_dev  MINDlarge_test  MINDlarge_train	MINDsmall_dev  MINDsmall_train
glove.840B.300d.txt
Generator.py  Hypers.py  Models.py  Preprocessing.py  Utils.py


In [5]:
!ls .

gdrive	Generator.py  Hypers.py  Models.py  Preprocessing.py  sample_data  Utils.py


In [6]:
from Hypers import *
from Utils import *
from Preprocessing import *
from Generator import *
from Models import *
from pathlib import Path
import os

project_path = '/content/gdrive/MyDrive/Dataset/MIND/'
data_root_path = os.path.join(project_path, '_data_')
embedding_path = os.path.join(project_path, '_embeddings_')

print(data_root_path)
print(embedding_path)


/content/gdrive/MyDrive/Dataset/MIND/_data_
/content/gdrive/MyDrive/Dataset/MIND/_embeddings_


In [7]:
!pip install nltk tensorflow dask pandas

In [8]:
# download punkt data for nltk
import nltk
# nltk_data_dir_path = os.path.join(project_path, '_nltk_data_')
# os.environ['NLTK_DATA'] = nltk_data_dir_path
nltk.download('punkt')#, download_dir=nltk_data_dir_path)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
print(data_root_path)
news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict = read_news(
  data_root_path,
)


news_title,news_vert,news_subvert,news_entity,news_content=get_doc_input(news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict)

/content/gdrive/MyDrive/Dataset/MIND/_data_
MINDlarge_train
MINDlarge_test
MINDlarge_dev
MINDsmall_train
MINDsmall_dev


In [10]:
title_word_embedding_matrix = load_matrix(embedding_path,word_dict)
content_word_embedding_matrix = load_matrix(embedding_path,content_dict)

In [11]:
def print_partial(d):
  print(dict(list(d.items())[0:4]))

print_partial(news)
print_partial(news_index)
print_partial(category_dict)
print_partial(subcategory_dict)

print(len(word_dict))
print(len(content_dict))

print(news_title.shape)
print(news_content.shape)

print(title_word_embedding_matrix.shape)
print(content_word_embedding_matrix.shape)



{'N88753': ['lifestyle', 'lifestyleroyals', ['the', 'brands', 'queen', 'elizabeth', ',', 'prince', 'charles', ',', 'and', 'prince', 'philip', 'swear', 'by'], ['Q80976', 'Q43274', 'Q9682'], ['shop', 'the', 'notebooks,', 'jackets,', 'and', 'more', 'that', 'the', 'royals', "can't", 'live', 'without.']], 'N45436': ['news', 'newsscienceandtechnology', ['walmart', 'slashes', 'prices', 'on', 'last-generation', 'ipads'], ['Q2796', 'Q483551'], ["apple's", 'new', 'ipad', 'releases', 'bring', 'big', 'deals', 'on', 'last', "year's", 'models.']], 'N23144': ['health', 'weightloss', ['50', 'worst', 'habits', 'for', 'belly', 'fat'], ['Q193583'], ['these', 'seemingly', 'harmless', 'habits', 'are', 'holding', 'you', 'back', 'and', 'keeping', 'you', 'from', 'shedding', 'that', 'unwanted', 'belly', 'fat', 'for', 'good.']], 'N86255': ['health', 'medical', ['dispose', 'of', 'unwanted', 'prescription', 'drugs', 'during', 'the', 'dea', "'s", 'take', 'back', 'day'], ['Q622899'], []]}
{'N88753': 1, 'N45436': 2,

In [12]:

train_session = read_train_clickhistory(news_index,data_root_path,'MINDlarge_train/behaviors.tsv')
dev_session = read_train_clickhistory(news_index,data_root_path,'MINDlarge_dev/behaviors.tsv')

train_user = parse_user(news_index,train_session)
dev_user = parse_user(news_index,dev_session)

train_sess, train_user_id, train_label = get_train_input(news_index,train_session)
dev_sess, dev_user_id, dev_label = get_train_input(news_index,dev_session)

In [13]:
# # Save preprocessing data as csv
# import pandas as pd
# import json

# def get_paths(folder):
#   train_session_path = os.path.join(project_path, '_data_', folder, 'train_session.csv')
#   train_user_path = os.path.join(project_path, '_data_', folder, 'train_user_id.csv')
#   train_label_path = os.path.join(project_path, '_data_', folder,  'train_label.csv')
#   train_click_path = os.path.join(project_path, '_data_',folder,  'train_clicks.csv')
#   train_meta_path = os.path.join(project_path, '_data_', folder, 'train-meta.json')

#   dev_session_path = os.path.join(project_path, '_data_', folder, 'dev_session.csv')
#   dev_user_path = os.path.join(project_path, '_data_', folder, 'dev_user_id.csv')
#   dev_label_path = os.path.join(project_path, '_data_', folder, 'dev_label.csv')
#   dev_click_path = os.path.join(project_path, '_data_', folder, 'dev_clicks.csv')
#   dev_meta_path = os.path.join(project_path, '_data_', folder, 'dev-meta.json')
#   return train_session_path, train_user_path, train_label_path, train_click_path, train_meta_path, dev_session_path, dev_user_path, dev_label_path, dev_click_path, dev_meta_path

# def save_processed_data_as_csv(folder):
#   train_session_path, train_user_path, train_label_path, train_click_path, train_meta_path, dev_session_path, dev_user_path, dev_label_path, dev_click_path, dev_meta_path = get_paths(folder)

#   train_sess_df = pd.DataFrame(train_sess)
#   train_sess_df.to_csv(train_session_path, index=False,header=False)

#   train_user_id_df = pd.DataFrame(train_user_id)
#   train_user_id_df.to_csv(train_user_path, index=False,header=False)

#   train_label_df = pd.DataFrame(train_label)
#   train_label_df.to_csv(train_label_path, index=False,header=False)

#   dev_sess_df = pd.DataFrame(dev_sess)
#   dev_sess_df.to_csv(dev_session_path, index=False,header=False)

#   dev_user_id_df = pd.DataFrame(dev_user_id)
#   dev_user_id_df.to_csv(dev_user_path, index=False,header=False)

#   dev_label_df = pd.DataFrame(dev_label)
#   dev_label_df.to_csv(dev_label_path, index=False,header=False)

#   train_click_df = pd.DataFrame(train_user['click'][train_user_id])
#   train_click_df.to_csv(train_click_path, index=False,header=False)

#   dev_click_df = pd.DataFrame(dev_user['click'][dev_user_id])
#   dev_click_df.to_csv(dev_click_path, index=False,header=False)


#   with open(train_meta_path, 'w') as f:
#     json.dump({ "impressionNum": train_label.shape[0] }, f)

#   with open(dev_meta_path, 'w') as f:
#     json.dump({ "impressionNum": dev_label.shape[0] }, f)

#   # return train_session_path, train_user_path, train_label_path, train_click_path, train_meta_path, dev_session_path, dev_user_path, dev_label_path, dev_click_path, dev_meta_path
# train_session_path, train_user_path, train_label_path, train_click_path, train_meta_path, dev_session_path, dev_user_path, dev_label_path, dev_click_path, dev_meta_path = get_paths('_small-processed_')
# # save_processed_data_as_csv('_small-processed_')
# # save_processed_data_as_csv('_processed_')


In [14]:
from Generator import NewsFetcher
news_fetcher = NewsFetcher(news_title,news_content,news_vert,news_subvert,news_entity)

In [15]:
train_generator = get_train_generator_2(news_fetcher,train_user['click'],train_user_id,train_sess,train_label,64)
val_generator = get_train_generator_2(news_fetcher,dev_user['click'], dev_user_id, dev_sess, dev_label, 64)
news_generator = get_hir_news_generator(news_fetcher,64)

In [16]:
print(len(train_generator))
print(len(val_generator))
# print(train_generator.label_reader.loc[100:120].compute().values)

res = val_generator[2000]

print(res[0][0].shape, res[0][1].shape, res[1].shape)

# train_generator[i] return i-th batch with format ((news_info, user_info), [labels])

# news_info.shape = (batch_size, train_sess.shape[1]=npratio+1, 87) -> represent the current clicked news and 4 not-clicked news
# user_info.shape = (batch_size, 50, 87) -> represent the clicked news before the current clicked news
# labels.shape = (batch_size, 5) -> 1 for clicked news, 0 for not-clicked news matched with news_info -> (1, 0, 0, 0, 0)

# test code
# gen_news_data, gen_labels = train_generator[0]
# gen_news_info, gen_user_info = gen_news_data

# print(gen_news_info.shape)
# print(gen_user_info.shape)
# print(gen_labels[0].shape)

52870
8982
(64, 5, 87) (64, 30, 87) (64, 5)


In [17]:
from keras import backend as K

K.clear_session()

model,news_encoder,user_encoder, = create_model(title_word_embedding_matrix,content_word_embedding_matrix,entity_dict,category_dict,subcategory_dict)

print("model")
model.summary()
print("news_encoder")
news_encoder.summary()
print("user_encoder")
user_encoder.summary()

model
Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_14 (InputLayer)       [(None, 30, 87)]             0         []                            
                                                                                                  
 input_15 (InputLayer)       [(None, 5, 87)]              0         []                            
                                                                                                  
 time_distributed_1 (TimeDi  (None, 30, 400)              4365399   ['input_14[0][0]']            
 stributed)                                               1                                       
                                                                                                  
 time_distributed_2 (TimeDi  (None, 5, 400)               4365399   ['input_15[0][0]'

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [18]:
import keras
import tensorflow as tf

checkpoint_dir = os.path.join(project_path, '_checkpoints_')
checkpoint_path = os.path.join(checkpoint_dir, 'cp.ckpt')

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

latest_weight = tf.train.latest_checkpoint(checkpoint_dir)

if latest_weight:
    model.load_weights(latest_weight)
    print("load weights from {}".format(latest_weight))
else:
    print("no weights found")


load weights from /content/gdrive/MyDrive/Dataset/MIND/_checkpoints_/cp.ckpt


In [19]:
cp_callback =keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit_generator(train_generator,epochs=3, verbose=True, validation_data=val_generator, callbacks=[cp_callback])

Epoch 1/3


<ipython-input-19-1cabea32117d>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=3, verbose=True, validation_data=val_generator, callbacks=[cp_callback])


52870/52870 [==============================] - ETA: 0s - loss: 1.3028 - acc: 0.5129
Epoch 1: saving model to /content/gdrive/MyDrive/Dataset/MIND/_checkpoints_/cp.ckpt
52870/52870 [==============================] - 7770s 147ms/step - loss: 1.3028 - acc: 0.5129 - val_loss: 1.4233 - val_acc: 0.4247
Epoch 2/3
   80/52870 [..............................] - ETA: 2:03:02 - loss: 1.2893 - acc: 0.5139

KeyboardInterrupt: ignored

In [ ]:
model_path = os.path.join(project_path, '_models_/model_small.keras')
model.save(model_path)